## Цель выявить закономерность между действиями и скрытыми картами оппонентов, предсказать силу комбинации
### На основе рекомендатеотных систем выявить типы оппонентов (колличесвто и их свойства - характерные действия относящие их к тому или иному типу\группе)
### На основе дерева решений и\или типа оппонентов предсказать силу комбинации
### Какие линии приносят больше фишек (в соотношении с ББ, обезразмеренный показатель), точнее при ШД каке линии самые частые (тут собьраны выигранные раздачи) 
### Какие флопы чаще выигрыввают
### По турнирно (фильтра ШД + раздачи с Хиро на ривере) в равных колл. раздачь, замерить колличесвто попаданий в готовые комбо по группам, посмотреть закономерности
 - Преобраование данных в числовой тип
 - Добавление новых признаков (размер банка, размер ставки в четвертях <=1/3,<=1/2,<=2/3,<=3/4,<=1,>1, процент активности\агрессии из признаков активности\агрессии)
 - https://github.com/julianandrews/flopferret/blob/master/flopferret/board_texture.py

In [1]:
import re
import pandas as pd
from datetime import datetime
import numpy as np

import eval7, pprint
from phevaluator import evaluate_cards
from poker.room.pokerstars import PokerStarsHandHistory
from poker.hand import Hand, Combo
from poker.constants import Action, Position
from poker import handhistory

import matplotlib.pyplot as plt
plt.style.use('dark_background') #стиль графиков

import sqlite3

import sentry_sdk

In [62]:
%%time

# Создали словарь
data_pd = {'Tournament':[], 'HandId': [], 'Level': [], 'ante':[], 'BB':[], 'Players': [], 'Board':[], 'SHOW_DOWN':[],'pflop_chip_rais':[], 'pflop_chip_calls':[], 'flop_chip_bets':[], 'flop_chip_raises':[],
           'flop_chip_calls':[], 'turn_chip_bets':[], 'turn_chip_raises':[], 'turn_chip_calls':[], 'river_chip_bets':[], 'river_chip_raises':[], 'river_chip_calls':[],   
            'pflop_rais':[], 'pflop_calls':[], 'pflop_allin':[], 'flop_bets':[], 'flop_raises':[], 'flop_calls':[], 
           'flop_allin':[], 'turn_bets':[], 'turn_raises':[], 'turn_calls':[], 'turn_allin':[], 'river_bets':[], 'river_raises':[], 'river_calls':[], 'river_allin':[]} 


# запрос из базы
conn = sqlite3.connect(r'C:\Users\artio\Documents\Holdem Manager 3\Databases\Bot_Praktik_2023.hmdb')
cur = conn.cursor() # создаём курсор для получения не всей базы а чтения по строчно

data_r = ''
for i, line in enumerate(cur.execute('select * from handhistories;')): #  limit 100
    data_r = data_r + line[3]
    if i > 50000:
        break

iterr = re.findall("PokerStars", data_r).count('PokerStars') # задаём колличесвто итераций
print(iterr)

data = ""
iter_SHOW = 0

for i in range(iterr+1): # фильтруем раздачи с ШД и без
    if data_r.split("PokerStars")[i].count('SHOW DOWN'):
        data = data + data_r.split("PokerStars")[i] + "\n\n\n"
        iter_SHOW += 1
    else:
        continue
print(iter_SHOW) 

# данные
result = re.findall(" Hand #(\d*): Tournament #(\d*), .(\d*)\+.(\d*) USD Hold'em No Limit - Level (\D*) \(\d*\/(\d*)\) - (\d{4}/\d+/\d+) (\d+\D\d+\D\d+) (\w*)", data) 
# (\w+) Hand #(\d*): Tournament #(\d*),(.*) Level (.*) \(.*\/(.*)\) - (\d{4}/\d+/\d+) (\d+\D\d+\D\d+) (\w*)
#Tables = re.findall('Table \'.*\' (.*)-max Seat', data)
#bains = re.findall('(\D\d+\.\d+\+\D\d+\.\d+)', data)
#HandHiros = re.findall("Dealt to .* \D(.. ..)\D", data)
Boards = re.findall("Board \[(.. .. .. .. ..)\]", data)
#preflop_raises_ = re.findall("raises", data) # "raises"g
SHOW_DOWNs = re.findall("showed \[(.. ..)\] and won", data)
#SHOW_DOWN.group(1)

# -------------------------  ПРЕФЛОП  ------------------------------

def preflop_acti(self):
    preflop_ = self
    action_preflop_raises = preflop_.group(1).count('raises')    
    action_preflop_calls = preflop_.group(1).count('calls')
    action_preflop_allin = preflop_.group(1).count('all-in')
    return action_preflop_raises, action_preflop_calls, action_preflop_allin

def street_raise_sum(self):
    street_ = self.group(1)    
    matches = re.findall(r'(?<=raises) (\d*) to (\d*)',street_)  
    n = 0
    sum_rais = []
    for i in enumerate(matches):
        rais = int(matches[n][1])
        sum_rais.append(rais)
        n += 1
    return sum(sum_rais)

def street_calls_sum(self):
    street_ = self.group(0)    
    matches = re.findall(r'(?<=calls) (\d*)',street_)  
    n = 0
    sum_calls = []
    for i in enumerate(matches):
        calls = int(matches[n])
        sum_calls.append(calls)
        n += 1
    return sum(sum_calls)

# -------------------------  ФЛОП  ------------------------------

def flop_acti(self):
    flop_ = self
    action_flop_bets = flop_.group(1).count('bets') # колличесвто
    action_flop_raises= flop_.group(1).count('raises')
    action_flop_calls = flop_.group(1).count('calls')
    action_flop_allin = flop_.group(1).count('all-in') 
    return action_flop_bets, action_flop_raises, action_flop_calls, action_flop_allin

def street_bets(self):
    street_ = self.group(1)
    matches = re.findall(r'(?<=bets) (\d*)',street_)
    if matches:
        return int(matches[0])
    else:
        return 0

# -------------------------  ТЁРН  ------------------------------

def turn_acti(self):
    turn_ = self
    action_turn_bets = turn_.group(1).count('bets')
    action_turn_raises= turn_.group(1).count('raises')
    action_turn_allin = turn_.group(1).count('all-in')
    action_turn_calls = turn_.group(1).count('calls')
    return action_turn_bets, action_turn_raises, action_turn_calls, action_turn_allin

# -------------------------  РИВЕР  ------------------------------

def river_acti(self):
    river_ = self
    action_river_bets = river_.group(1).count('bets')
    action_river_raises= river_.group(1).count('raises')
    action_river_allin = river_.group(1).count('all-in')
    action_river_calls = river_.group(1).count('calls')
    return action_river_bets, action_river_raises, action_river_calls, action_river_allin

#----ПРЕФЛОП------
preflop_raisess = []
preflop_callss = [] 
preflop_allins = []
preflop_raisess_sums =[]
preflop_calls_sums =[]
antes = []

#----ФЛОП------
flop_betss = []  
flop_raisess = [] 
flop_callss = []
flop_allins = []
flop_chip_betss = []  
flop_chip_callss = []
flop_chip_raisess = [] 

#------ТЁРН--------
turn_betss = []  
turn_raisess = [] 
turn_callss = []
turn_allins = []
turn_chip_betss = []  
turn_chip_raisess = [] 
turn_chip_callss = []

#-------РИВЕР---------
river_betss = []  
river_raisess = [] 
river_callss = []
river_allins = []
river_chip_betss = []  
river_chip_raisess = [] 
river_chip_callss = []


for y in range(iter_SHOW):
    data_n = data.split("\r\n\n\n\n")[y] # разбивает сплошной текст на раздачи
    # print(repr(data_n))
# -------------------------  ПРЕФЛОП  ------------------------

    ante = re.findall('ante (\d*)', data_n)[:1]
    antes.append(ante[0][0:])

    preflop_ = re.search("HOLE CARDS \*{3}(.*)\*{3} FLOP", data_n, re.DOTALL)
    preflop_raisess.append(preflop_acti(preflop_)[0])
    preflop_callss.append(preflop_acti(preflop_)[1])
    preflop_allins.append(preflop_acti(preflop_)[2])
    preflop_raisess_sums.append(street_raise_sum(preflop_))
    preflop_calls_sums.append(street_calls_sum(preflop_))
    
# ------------------------- ФЛОП  ------------------------

    flop_ = re.search("FLOP \*{3} \[.*\](.*)\*{3} TU", data_n, re.DOTALL)
    flop_betss.append(flop_acti(flop_)[0])
    flop_raisess.append(flop_acti(flop_)[1])
    flop_callss.append(flop_acti(flop_)[2])
    flop_allins.append(flop_acti(flop_)[3])    
    flop_chip_betss.append(street_bets(flop_)) 
    flop_chip_callss.append(street_calls_sum(flop_))
    flop_chip_raisess.append(street_raise_sum(flop_))

# -------------------------  ТЁРН   ------------------------

    turn_ = re.search("TURN \*{3} \[.*\] \[.*\](.*)\*{3} RI", data_n, re.DOTALL)
    turn_betss.append(turn_acti(turn_)[0])
    turn_raisess.append(turn_acti(turn_)[1])
    turn_callss.append(turn_acti(turn_)[2])
    turn_allins.append(turn_acti(turn_)[3])
    turn_chip_betss.append(street_bets(turn_))
    turn_chip_callss.append(street_calls_sum(turn_))
    turn_chip_raisess.append(street_raise_sum(turn_))

# -------------------------  РИВЕР  ------------------------

    river_ = re.search("RIVER \*{3} \[.*\] \[.*\](.*)\*{3} SHOW", data_n, re.DOTALL)
    river_betss.append(turn_acti(river_)[0])
    river_raisess.append(turn_acti(river_)[1])
    river_callss.append(turn_acti(river_)[2])
    river_allins.append(turn_acti(river_)[3])
    river_chip_betss.append(street_bets(river_))
    river_chip_callss.append(street_calls_sum(river_))
    river_chip_raisess.append(street_raise_sum(river_))    

    
# Считаем сколко фишек
s = re.findall('Seat (\d).*chips', data)
s = [int(x) for x in s]
seats = []
length = 1
for i in range(len(s)-1):
    if s[i+1] < s[i]:
        seats.append(length)
        length = 1
    else:
        length += 1                     
seats.append(length)

# ЗАполнЯем словарь данными
for r, seat, Board ,preflop_raises, preflop_calls, preflop_allin, flop_bets, flop_raises, flop_calls, flop_allin, \
    turn_bets, turn_raises, turn_calls, turn_allin, river_bets, river_raises, river_calls, river_allin, SHOW_DOWN, preflop_raisess_sum, \
        preflop_calls_sum, ante, flop_chip_bets, flop_chip_calls, flop_chip_raises, turn_chip_bets, turn_chip_raises, turn_chip_calls, river_chip_bets, \
            river_chip_raises, river_chip_calls \
                in zip(result, seats, Boards, preflop_raisess, preflop_callss, preflop_allins, flop_betss, flop_raisess, flop_callss, 
                       flop_allins, turn_betss, turn_raisess, turn_callss, turn_allins, river_betss, river_raisess, river_callss, river_allins, SHOW_DOWNs, 
                            preflop_raisess_sums, preflop_calls_sums, antes, flop_chip_betss, flop_chip_callss, flop_chip_raisess, turn_chip_betss, turn_chip_raisess,
                               turn_chip_callss, river_chip_betss, river_chip_raisess, river_chip_callss): 

    #data_pd['TypeOfRoom'].append(r[0])
    data_pd['HandId'].append(r[0])
    data_pd['Tournament'].append(r[1])
    # data_pd['info'].append(r[3])
    data_pd['Level'].append(r[4])
    data_pd['BB'].append(r[5])
    #data_pd['Date'].append(r[6])
    #data_pd['Hour'].append(r[7])
    #data_pd['TimeZone'].append(r[8])
    data_pd['ante'].append(ante)
    #data_pd['Table'].append(table) #table
    data_pd['Players'].append(seat)
    #data_pd['bain'].append(bain)
    #data_pd['HandHiro'].append(handHiro)
    data_pd['Board'].append(Board)
    data_pd['pflop_calls'].append(preflop_calls)
    data_pd['pflop_rais'].append(preflop_raises)
    data_pd['pflop_allin'].append(preflop_allin)
    data_pd['flop_bets'].append(flop_bets)
    data_pd['flop_raises'].append(flop_raises)
    data_pd['flop_calls'].append(flop_calls)
    data_pd['flop_allin'].append(flop_allin)
    data_pd['turn_bets'].append(turn_bets)
    data_pd['turn_raises'].append(turn_raises)
    data_pd['turn_calls'].append(turn_calls)
    data_pd['turn_allin'].append(turn_allin)
    data_pd['river_bets'].append(river_bets)
    data_pd['river_raises'].append(river_raises)
    data_pd['river_calls'].append(river_calls)
    data_pd['river_allin'].append(river_allin)
    data_pd['pflop_chip_rais'].append(preflop_raisess_sum)
    data_pd['pflop_chip_calls'].append(preflop_calls_sum)
    data_pd['flop_chip_bets'].append(flop_chip_bets)
    data_pd['flop_chip_raises'].append(flop_chip_raises)
    data_pd['flop_chip_calls'].append(flop_chip_calls)
    data_pd['turn_chip_bets'].append(turn_chip_bets)
    data_pd['turn_chip_raises'].append(turn_chip_raises)
    data_pd['turn_chip_calls'].append(turn_chip_calls)
    data_pd['river_chip_bets'].append(river_chip_bets)
    data_pd['river_chip_raises'].append(river_chip_raises)
    data_pd['river_chip_calls'].append(river_chip_calls)
    data_pd['SHOW_DOWN'].append(SHOW_DOWN) 
                                    

# Создаём ДатаФрейм
df = pd.DataFrame(data_pd)

In [65]:
df.head()

,Tournament,HandId,Level,ante,BB,Players,Board,SHOW_DOWN,pflop_chip_rais,pflop_chip_calls,...,flop_calls,flop_allin,turn_bets,turn_raises,turn_calls,turn_allin,river_bets,river_raises,river_calls,river_allin
0,3837356897,1546300121,I,250,2000,7,Ks 9s Ts As 3s,Jd 4h,0,2000,...,0,0,0,0,0,0,0,0,0,0
1,3837356897,1546300638,I,250,2000,7,7c Kh 5s 6h 3s,4d Ac,13125,12125,...,1,0,1,0,1,0,0,0,0,0
2,3837356897,1546301654,I,250,2000,7,7d 6h 8h Qs Qh,Ks 8h,327543,181332,...,0,0,0,0,0,0,0,0,0,0
3,3837356897,1546303904,I,250,2000,7,Ac 7h 8h 6s Kd,2h Ah,97373,89373,...,0,0,0,0,0,0,0,0,0,0
4,3837356897,1546304725,I,250,2000,7,5s As 6s 3c 9d,6d As,8750,14500,...,2,0,1,0,2,0,1,0,1,2


In [63]:
# Сохранение датафрейма
df.to_csv('file_16486.csv', header=True, index=True)

In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15237 entries, 0 to 15236
Data columns (total 34 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Tournament         15237 non-null  object
 1   HandId             15237 non-null  object
 2   Level              15237 non-null  object
 3   ante               15237 non-null  object
 4   BB                 15237 non-null  object
 5   Players            15237 non-null  int64 
 6   Board              15237 non-null  object
 7   SHOW_DOWN          15237 non-null  object
 8   pflop_chip_rais    15237 non-null  int64 
 9   pflop_chip_calls   15237 non-null  int64 
 10  flop_chip_bets     15237 non-null  int64 
 11  flop_chip_raises   15237 non-null  int64 
 12  flop_chip_calls    15237 non-null  int64 
 13  turn_chip_bets     15237 non-null  int64 
 14  turn_chip_raises   15237 non-null  int64 
 15  turn_chip_calls    15237 non-null  int64 
 16  river_chip_bets    15237 non-null  int64

### Размер ставок от банка


In [ ]:
list(df.columns.values)

In [ ]:
df[['ante','BB',
    'pflop_chip_rais',
     'pflop_chip_calls',
     'flop_chip_bets',
     'flop_chip_raises',
     'flop_chip_calls',
     'turn_chip_bets',
     'turn_chip_raises',
     'turn_chip_calls',
     'river_chip_bets',
     'river_chip_raises',
     'river_chip_calls']].head()

In [ ]:
df['Flop_group'] = df.Flop.map(flop_flush)
df['Flop_group']



### Структура флопа

In [ ]:
# выделяем флоп в одельный столбец
df['Flop'] = df['Board'].str.extract(r'(.{8})', expand=False)
df['Flop']

In [ ]:
# %load flop_classification.py
from range import Range
import numpy as np

deck = ['A', 'K', 'Q', 'J', 'T', '9', '8', '7', '6', '5', '4', '3', '2']
deck2 = ['2', '3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q', 'K', 'A']
suit = ['c', 'd', 'h', 's']

A = ['A']
HIGH = ['T', 'J', 'Q', 'K']
MED = ['6', '7', '8', '9']
LOW = ['2', '3', '4', '5']
group = [[A, 'A'], [HIGH, "H"], [MED, "M"], [LOW, "L"]]

paired = ["TRIPS", "PAIRED", "UNPAIRED"]  # Paired types
flush = ["RAINBOW", "TWO-TONE", "MONOTONE"]  # Flushes Type
STR = ["OESD", "GUTSHOT", "STRAIGHT", "NONE"]  # Straights Type


class Flop:
    # create list of flop, card in flop, and suit in flop
    flop = None
    flop_card = None
    flop_suit = None
    group = None
    paired = None
    flush = None
    strit = None

    # init flop with input flop
    def __init__(self, text):
        self.flop = []
        self.flop_card = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        self.flop_suit = [0, 0, 0, 0]
        self.set_flop(text)
        self.card_and_suit_count()
        self.set_group()
        self.set_fullhouse()
        self.set_flush()
        self.set_strit()

    # set flop
    def set_flop(cls, text):
        if ' ' in text:
            temp = text.split(' ')
        else:
            temp = [text[0:2], text[2:4], text[4:6]]
        cls.flop = temp.copy()

    # count card and suit
    def card_and_suit_count(cls):
        for card in cls.flop:
            temp_card = card[0]
            temp_suit = card[1]
            cls.flop_card[deck.index(temp_card)] += 1
            cls.flop_suit[suit.index(temp_suit)] += 1

    # set cls.group ex. AHM AMM MML
    def set_group(cls):
        cards = cls.flop
        group_cards = ""
        for card in cards:
            for group_type in group:
                if card[0] in group_type[0]:
                    group_cards += group_type[1]
                    continue
        cls.group = group_cards

    # set cls.paired ex. PAIRED UNPAIRED TRIPS
    def set_fullhouse(cls):
        if 3 in cls.flop_card:  # TRIPS
            cls.paired = paired[0]
        elif 2 in cls.flop_card:  # PAIRED
            cls.paired = paired[1]
        else:  # UNPAIRED
            cls.paired = paired[2]

    # set cls.flush ex. MONOTONE TWO-TONE RAINBOW
    def set_flush(cls):
        if 3 in cls.flop_suit:  # MONOTONE
            cls.flush = flush[2]
        elif 2 in cls.flop_suit:  # TWO-TONE
            cls.flush = flush[1]
        else:  # RAINBOW
            cls.flush = flush[0]

    # выводит двухсторонку, гатшот и стрит
    def set_strit(cls):
        index = []
        for n in cls.flop:
            for i in deck2:
                i = str(deck2.index(n[:1])).zfill(2)
            index.append(i)
        index.sort()
        index_int = [int(x) for x in index]
        if (index_int[0] == 0 and index_int[1] == 1 and index_int[2] == 12) or \
                (index_int[0] == 10 and index_int[1] == 11 and index_int[2] == 12):
            cls.strit = STR[2]
        elif (index_int[2] - index_int[1]) == 1 and \
                (index_int[1] - index_int[0]) == 1:
            cls.strit = STR[0]
        elif ((index_int[2] - index_int[1]) == 2 and \
              (index_int[1] - index_int[0]) == 1) or \
                ((index_int[2] - index_int[1]) == 1 and \
                 (index_int[1] - index_int[0]) == 2) or \
                (index_int[0] == 0 and index_int[1] == 2 and index_int[2] == 12) or \
                (index_int[0] == 1 and index_int[1] == 2 and index_int[2] == 12):
            cls.strit = STR[1]
        else:
            cls.strit = STR[3]

            # return flop card only (no suit)

    def get_flop_card(cls):
        return cls.flop[0][0] + cls.flop[1][0] + cls.flop[2][0]

    # print Flop ex. AcTc5d
    def __str__(self):
        return self.flop[0] + self.flop[1] + self.flop[2]


if __name__ == '__main__':
    t = Flop('3c 2c 4c')
    print(t)
    print(t.group)
    print(t.paired)
    print(t.flush)
    print(t.strit)


In [ ]:

for i in df['Flop']:
    print(flop_class(i))


In [ ]:
df['Flop_group'] = df.Flop.map(flop_flush)
df['Flop_group']

# df['Flop_group'] = df['Flop'].apply(flop_flush)
# df['Flop_group']

## Преобразуем данные

In [ ]:
# cудаление столбца

df.pop('Flop_gro')

In [ ]:
df

In [ ]:
date = data_pd['Date']
time = data_pd['Hour']

dataf["Date"] = pd.to_datetime(data_pd['Date'])
dataf["Time"] = pd.to_timedelta(data_pd['Hour'])

dataf["DateTime"] = dataf["Date"] + dataf["Time"]
dataf["DateTime"]

# filter by single day
dataf_m = dataf[dataf['DateTime'].dt.strftime('%d') == '06']

dataf.Tournament = dataf.Tournament.astype('int64')
dataf = dataf.astype({'HandId': np.int64, 'Tournament': np.int64})

In [ ]:
dataf_m

In [ ]:
nov_mar_series = pd.Series(pd.date_range("2013-11-15", "2014-03-15"))
#create timestamp without year
nov_mar_no_year = nov_mar_series.map(lambda x: x.strftime("%m-%d"))
nov_mar_no_year

In [ ]:
# #add a yearless timestamp to the dataframe
# df["no_year"] = df['nov_mar_series'].map(lambda x: x.strftime("%m-%d"))
# no_year_mask = df['no_year'].isin(nov_mar_no_year)
# df[no_year_mask]

## Фильтруем турниры по числу рук выше среднего (> mean)

In [ ]:
data_filter = dataf[['Tournament', 'HandId','DateTime', 'Time', 'HandHiro', 'Board']]
data_filter.head()

In [ ]:
# data_filter.groupby(['Tournament']).agg({'HandId': "count"}).reset_index()

In [ ]:
#  Фильтруйте value_counts с помощью isin  https://softhints.com/pandas-how-to-filter-results-of-value_counts/
df = data_filter[data_filter['Tournament'].isin(data_filter['Tournament'].
                                                 value_counts()[data_filter['Tournament'].value_counts()>100].index)]
df

In [ ]:
#  Используйте group by и lambda для имитации фильтра для value_counts()
#df = data_filter.groupby('Tournament').filter(lambda x: len(x) > (data_filter['Tournament'].value_counts().mean()))


In [ ]:
df.Tournament.value_counts()

In [ ]:
# Prepare Data
df_plot = df.groupby('Tournament').size()

# Make the plot with pandas
df_plot.plot(kind='pie', subplots=True, figsize=(8, 8))
plt.title("Колличество турниров")
plt.ylabel("")
plt.show()

In [ ]:
# объеденяем руку и борд, выводим название комбинации

# df['hand'] = df['HandHiro'] + " " + df['Board']
# # hand = df['hand'].values[7].split()
# # print(hand)

# hand_ev = [eval7.Card(s) for s in (df['hand'].values[7].split())]
# rang = eval7.evaluate(hand_ev)   # ранг комбинации
# # print(rang)

# eval7s = eval7.handtype(rang)  #  название комбинации

# df['rang'] = eval7.evaluate(hand_ev)
# df['eval7s'] =  eval7.handtype(rang)

In [ ]:
# def new_type(row):
#     if row.traffic_source.lower() in ('yandex', 'google'):
#         return 'organic'
#     elif row.traffic_source.lower() in ('paid', 'email'):
#         if row.region.lower() == 'russia':
#             return 'ad'
#         else:
#             return 'other'

In [ ]:
# log['source_type'] = log.apply(new_type, axis = 1)
# log.head(12)

In [ ]:
# log.loc[log.traffic_source.isin(['yandex', 'google']), 'traffic_type'] = 'organic' 

In [ ]:
df['hand_rang'] = df['HandHiro'] + " " + df['Board']
df['hand_rang']

print(type(df['hand_rang']), type(df['hand_rang'].values[7].split()))
df['hand_rang'].values[0].split()

In [ ]:
# df['hand_rang'].to_list()
# df['hand_rang'].tolist()
# np.array(df['hand_rang'].tolist())
# Ls = list(df['hand_rang'])
ls = df['hand_rang'].str.split()
ls

In [ ]:
# напишем функцию получения ранг комбинации

def evaluate_(hand):
    """
    На вход получает карты, на выход 
    число ранг по eval7
    """
    hand_ev = [eval7.Card(s) for s in (hand)]
    return eval7.evaluate(hand_ev),hand_ev

a = ('Jh', 'Td', '8h', '8c', '7c', '3h', '6d')
evaluate_(a)

In [ ]:
# напишем функцию получения названия ранга

def handtype_(rang):
    """
    На вход получает ранг, на выход 
    название комбинации
    """
    return eval7.handtype(rang)

handtype_(17209424)

In [ ]:
# Применим функцию к столбцу hand  и запишем в новый столбец rang_namber

df['rang_namber'] = df['hand_rang'].str.split().apply(evaluate_)
df['handtypes'] = df['rang_namber'].apply(handtype_)
df[['rang_namber', 'handtypes','HandHiro', 'Board']].head(20)
# df['filter'] = df.loc[df.handtypes.str.contains('Two Pair', 'Trips', 'Straight', 'Full House', 'Quads', 'Straight Flush'), 'rang_namber']

In [ ]:
# df.sort_values(by=['handtypes', 'rang_namber'])

In [ ]:
# df.groupby('handtypes')['rang_namber'].value_counts()

### Создадим сводную таблицу, что-бы проверить наличие аномалий

In [ ]:
# filter by single day
dataf_m = dataf[dataf['DateTime'].dt.strftime('%d') == '06']

In [ ]:
df[['Tournament', 'handtypes', 'rang_namber']].head()  # применим фильтер по столбцам

In [ ]:
pd.pivot_table(df, index=["Tournament", "handtypes"])

In [ ]:
# явно определив столбцы, которые нам нужны,
pd.pivot_table(df,
               index=["Tournament", "handtypes"],
               values=["rang_namber"],
               aggfunc= len)   # aggfunc=[np.mean, len])

In [ ]:
df.Tournament.value_counts()

In [ ]:
# df['Time'].value_counts()

In [ ]:
# Prepare Data
df_plot = df.groupby('handtypes').size()

# Make the plot with pandas
df_plot.plot(kind='area', subplots=True, figsize=(8, 8))
plt.title("Колличество комбинаций")
plt.ylabel("")
plt.show()

In [ ]:
# df['DateTime'] = df['DateTime'][(df['DateTime'] > '2018-10-01 20:00:00') & (df['DateTime'] < '2022-10-02 00:00:00')]
# df['DateTime']


In [ ]:
# отфильтруем комбинации "старшая карта" и "пара"
df['no_par'] = df['rang_namber'][df['rang_namber'] > 50000000] #[((df['rang_namber'] > 50000000) & (df['rang_namber'] < 60000000))]
df = df.dropna()
df[['Time', 'no_par', 'handtypes']].head(10)
# df['filter_Two Pair'] = df['handtypes'][df['handtypes'] == 'Two Pair']
# df['filter_Two Pair']

In [ ]:
df.isnull().sum()

In [ ]:
# преобразуем время во временной ряд

In [ ]:


idx = df['Time']
 # datetime.timedelta(seconds=24*60*60).total_seconds()

# second = df['DateTime'].dt.total_seconds()
# second

# idx = pd.date_range(df['DateTime'], periods=5, freq="H")
# idx
tim = pd.Series(range(len(idx)), index=idx)

# ts.resample("2H").mean()
tim

In [ ]:
# df = df.set_index("Time")
# df

In [ ]:

import matplotlib.pyplot as plt
 
# Using a inbuilt style to change
# the look and feel of the plot
plt.style.use("classic")
 
# setting figure size to 12, 10
plt.figure(figsize=(12, 10))
 
# Labelling the axes and setting
# a title
plt.xlabel("Time")
plt.ylabel("Values")
plt.title("Sample Time Series Plot")
 
# plotting the "A" column alone
plt.plot(df["handtypes"])

In [ ]:
df.plot(subplots=True, figsize=(12, 15))

In [ ]:
# # # idx = pd.date_range('2019-01-01', periods=48, freq='H')
# # x = pd.Series(range(len(idx)), index=idx)
# y = tim
# # add 'hour' and 'day' columns in the dataframe
# d = {'x': x, 'y': y, 'hour': idx.hour, 'day': idx.day}
# df = pd.DataFrame(data=d)
# # use 'hour' as x axis to plot, and 'day' as marker color
# df.plot.scatter(x='hour', y='y', c='day', colormap='rainbow')
# plt.show()

In [ ]:
data.dtypes

In [ ]:
# установитьdatetimeОбъект в качестве индекса кадра данных
data = data.set_index('DateTime')
data.head()

In [ ]:

# data.plot(grid=True)

data['filter'].plot(kind='bar')

# import matplotlib.pyplot as plt
# from matplotlib.dates import DateFormatter
# # Группировка за месяц
# df_month = data.resample("M").mean()

# fig, ax = plt.subplots(figsize=(10, 6))
# ax.xaxis.set_major_formatter(DateFormatter('%Y-%m'))
# ax.bar(
#     df_month['2018':].index, 
#     df_month.loc['2016':, "filter"], 
#     width=25, align='center')

In [ ]:

data.plot(kind='scatter', 
        x='DateTime', 
        y='filter', 
        title='Связь между жёсткостью воды и средней годовой смертностью')